# Neural Question Answering Model for SFU-related Topics

Simon Fraser University CMPT825 Natural Language Processing 

Python notebook Created By Chi Chung (Cyrus) Chan, Weijia Li , Peng Yi

Date: 8 Dec, 2019

Email: {cyrusc, wla171, pengy} @ sfu.ca


You will need these library for the notebook

In [46]:
!pip install -r requirements.txt

  Using cached https://files.pythonhosted.org/packages/70/1a/364556102943cacde1ee00fdcae3b1615b39e52649eddbf54953e5b144c9/transformers-2.2.1-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/1f/8e/ed5364a06a9ba720fddd9820155cc57300d28f5f43a6fd7b7e817177e642/sacremoses-0.0.35.tar.gz
     |████████████████████████████████| 133kB 2.1MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/e8/cf/7089b87fdae8f47be81ce8e2e6377b321805c4648f2eb12fbd2987388dac/sentencepiece-0.1.83-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/fc/1d/13cc7d174cd2d05808abac3f5fb37433e30c4cd93b152d2a9c09c926d7e8/regex-2019.11.1.tar.gz
  Using cached https://files.pythonhosted.org/packages/16/8a/1fc3dba0c4923c2a76e1ff0d52b305c44606da63f718d14d3231e21c51b0/s3transfer-0.2.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/83/94/7179c3832a6d45b266ddb2aac329e101367fbdb11f425f13771d27f225bb/jmespath-0.9.4-py2.py3-non

In [47]:
import json, re
import utils_squad
from utils_squad import *

I1208 04:26:13.550851 140371782047552 file_utils.py:40] PyTorch version 1.3.0 available.


 ## 1. Motivation

In this project, we want to improve an open-domain question answering (QA) system based on Google’s Natural Questions dataset.  Before 2017, the QA system mainly focused on LSTM (RNN) approach; however, RNN has lower performance on prediction speed and costs more training time.  In 2017 Google released a transformer based model BERT,Bidirectional Transformers, which improves overall performance, accuracy and fine tune appoarch for natural language processing task in different domain. In 2019, Google released another improvement model ALBERT, which has lower diemeion on embedding space and used less parameters based on BERT model. Also, DistilBERT model
 

In this project, we aim to answer the following research questions:

1. Can neural QA model trained on generic dataset perform well at answering domain specific questions? 
2. Do model performance drop consistently across all algorithms, if any performance drop is observed?
3. Can ensemble models improve the overall performance?

## 2.  Data Introduction

There are two public datasets commonly used for QA task, Stanford Question Answering Dataset (SQuAD)in section 2.1 [Reference b] and Google Natural Question Dataset(NQ) in section 2.2 [Reference c]. And, we are also interested to test our model with question related to our school, Simon Fraser University (SFU) in section 2.3; therefore, we would also create some related dataset by ourselves.

### 2.1 SQuAD v1.1 Dataset

#### 2.1.1 SQuAD v1.1 Data Source

We would like to use SQuAD v1.1 dataset, instead of v2.0 cause of the time and computatinal hardware resource constraint for the project. By using smaller dataset, we can reduce the time on training data.

In [5]:
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json

--2019-12-08 03:52:13--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.110.153, 185.199.109.153, 185.199.111.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.110.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30288272 (29M) [application/json]
Saving to: ‘train-v1.1.json.2’

train-v1.1.json.2   100%[===================>]  28.88M  31.2MB/s    in 0.9s    

2019-12-08 03:52:14 (31.2 MB/s) - ‘train-v1.1.json.2’ saved [30288272/30288272]

--2019-12-08 03:52:14--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.110.153, 185.199.109.153, 185.199.111.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.110.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4854279 (4.6M) [application/json]
Saving to: ‘dev-v1.1.json.2’

dev-v1.1.json.

#### 2.1.2 SQuAD Data Sample 

In [27]:
with open('train-v1.1.json', "r", encoding='utf-8') as reader:
    squad_data = json.load(reader)["data"]
    def is_whitespace(c):
        if c == " " or c == "\t" or c == "\r" or c == "\n" or ord(c) == 0x202F:
            return True
        return False
    for entry in squad_data:
        for paragraph in entry["paragraphs"]:
            paragraph_text = paragraph["context"]
            squad_tokens = []
            prev_is_whitespace = True
            for c in paragraph_text:
                if is_whitespace(c):
                    prev_is_whitespace = True
                else:
                    if prev_is_whitespace:
                        squad_tokens.append(c)
                    else:
                        squad_tokens[-1] += c
                    prev_is_whitespace = False
            for qa in paragraph["qas"]:
                squad_question_text = qa["question"]
                squad_answer_text = qa["answers"][0]["text"]
                break

        break
    
squad_context_text = ' '.join(squad_tokens)

print("First question-answer pair: \n")
print("Question: " + str(squad_question_text) + "\n")
print("Context: " + str(squad_context_text) + "\n")
print("Answers: " + str(squad_answer_text))


First question-answer pair: 

Question: Which Secretary of State attended Notre Dame?

Context: Notre Dame alumni work in various fields. Alumni working in political fields include state governors, members of the United States Congress, and former United States Secretary of State Condoleezza Rice. A notable alumnus of the College of Science is Medicine Nobel Prize winner Eric F. Wieschaus. A number of university heads are alumni, including Notre Dame's current president, the Rev. John Jenkins. Additionally, many alumni are in the media, including talk show hosts Regis Philbin and Phil Donahue, and television and radio personalities such as Mike Golic and Hannah Storm. With the university having high profile sports teams itself, a number of alumni went on to become involved in athletics outside the university, including professional baseball, basketball, football, and ice hockey players, such as Joe Theismann, Joe Montana, Tim Brown, Ross Browner, Rocket Ismail, Ruth Riley, Jeff Samardz

### 2.2 Google NQ Dataset

Natural Questions (NQ) contains 307,372 training examples which are real user questions issued to Google search, and answers found from Wikipedia by annotators. The data is designed for the training and evaluation of automatic question answering systems.

#### 2.2.1 NQ Data Source

In [28]:
%%bash
pip install gsutil

Download train and test dataset from Kaggle

In [29]:
!gsutil -m cp -R gs://natural_questions/v1.0-simplified/simplified-nq-train.jsonl.gz .
!unzip simplified-nq-train.jsonl.zip

Copying gs://natural_questions/v1.0-simplified/simplified-nq-train.jsonl.gz...
Resuming download for ./simplified-nq-train.jsonl.gz component 0                
Resuming download for ./simplified-nq-train.jsonl.gz component 1
Resuming download for ./simplified-nq-train.jsonl.gz component 2
Resuming download for ./simplified-nq-train.jsonl.gz component 3
^C[0/1 files][  2.0 GiB/  4.4 GiB]  45% Done  34.4 MiB/s ETA 00:01:11           
Exception in UIThread: 
Caught CTRL-C (signal 2) - exiting
Killed
unzip:  cannot find or open simplified-nq-train.jsonl.zip, simplified-nq-train.jsonl.zip.zip or simplified-nq-train.jsonl.zip.ZIP.


Because of the time and computatinal hardware resource constraint, we only consider finding short answer from the context in the dataset, which allows us to reduce the training time. And, we use the only first answer from the documents, there for the total number of question pairs are 106,926.

In [10]:
 # function for filter any question pair without short answer in Google NQ
train_input_file = 'simplified-nq-train.jsonl'
output_file = 'nq-short-train.json'
count = 0
with open(output_file, 'w') as new_json:
    with open(train_input_file, "r", encoding='utf-8') as reader:
        for line in reader:
            input_data = json.loads(line)
            short_answer = input_data['annotations'][0]['short_answers']
            # Skipping data with no short answer
            if(len(short_answer) < 1):
                continue
            new_json.write(line)
            count = count + 1
print("Number of short answer examples: " + str(count))

Number of short answer examples: 106926


#### 2.2.2 NQ Data Sample


As the pretrained weight for BERT are designed for maximum 512 tokens sequence input, we will use the long answer as the context in the project, which allows us to fill the input data which below the limilation.

In [14]:
with open('nq-short-train.json', "r", encoding='utf-8') as reader:
    def is_whitespace(c):
        if c == " " or c == "\t" or c == "\r" or c == "\n" or ord(c) == 0x202F:
            return True
        return False
    for line in reader:
        nq_data = json.loads(line)
        nq_tokens = []
        prev_is_whitespace = True
        for c in nq_data["document_text"]:
            if is_whitespace(c):
                prev_is_whitespace = True
            else:
                if prev_is_whitespace:
                    nq_tokens.append(c)
                else:
                    nq_tokens[-1] += c
                prev_is_whitespace = False
        break
long_answer = nq_data["annotations"][0]["long_answer"]
nq_context_text = ' '.join(nq_tokens[long_answer["start_token"]:long_answer["end_token"]])
short_answers = nq_data["annotations"][0]["short_answers"][0]
short_answers_text = ' '.join(nq_tokens[short_answers["start_token"]:short_answers["end_token"]])

print("First question-answer pair: \n")
print("Question: " + str(nq_data["question_text"]) + "\n")
print("Context: " + str(nq_context_text) + "\n")
print("Answers: " + str(short_answers_text))

First question-answer pair: 

Question: which is the most common use of opt-in e-mail marketing

Context: <P> A common example of permission marketing is a newsletter sent to an advertising firm 's customers . Such newsletters inform customers of upcoming events or promotions , or new products . In this type of advertising , a company that wants to send a newsletter to their customers may ask them at the point of purchase if they would like to receive the newsletter . </P>

Answers: a newsletter sent to an advertising firm 's customers


### 2.3 SFU-Related Test Dataset

#### 2.3.1 SFU Dataset Source

As we are interested on testing the model with SFU related question, we manually wrote 100 of question and answer pairs which are closely related to the SFU community (which will not be part of training data). The questions and answer pair sources are from Wikipedia. We use this dataset to test the more general QA model's performance on models later on.

In [24]:
!wget --no-check-certificate -O sfu.json "https://onedrive.live.com/download?cid=1ED3E57B6F717CEC&resid=1ED3E57B6F717CEC%21123637&authkey=ALCz8bDG7lIcuQQ"

--2019-12-08 04:09:44--  https://onedrive.live.com/download?cid=1ED3E57B6F717CEC&resid=1ED3E57B6F717CEC%21123637&authkey=ALCz8bDG7lIcuQQ
Resolving onedrive.live.com (onedrive.live.com)... 13.107.42.13
Connecting to onedrive.live.com (onedrive.live.com)|13.107.42.13|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://o8g3cq.by.files.1drv.com/y4mOCOMsuXTFwgaEzanaz8FRj0r8JznSBGWIsCIQExebnEm_fonBJJuYX_FQm7HSKEl7vJV1crzXN-NYFVaOZ4iptefsmWrsKWuI3njPh3uuCJhF1TWlVQ8tkXAnYDJDrFnMFk4LRB1AQrM8hUTsaJixMAF635-d4S0d5FELfmXTKy78L5kNvxBPzclXb0Fg3sba38XQ8fwDC6TwlgHxbHNKA/sfu.json?download&psid=1 [following]
--2019-12-08 04:09:45--  https://o8g3cq.by.files.1drv.com/y4mOCOMsuXTFwgaEzanaz8FRj0r8JznSBGWIsCIQExebnEm_fonBJJuYX_FQm7HSKEl7vJV1crzXN-NYFVaOZ4iptefsmWrsKWuI3njPh3uuCJhF1TWlVQ8tkXAnYDJDrFnMFk4LRB1AQrM8hUTsaJixMAF635-d4S0d5FELfmXTKy78L5kNvxBPzclXb0Fg3sba38XQ8fwDC6TwlgHxbHNKA/sfu.json?download&psid=1
Resolving o8g3cq.by.files.1drv.com (o8g3cq.by.files.1drv.com)... 1

#### 2.3.2 SFU Dataset Sample

In [39]:
with open('sfu.json', "r", encoding='utf-8') as reader:
    sfu_data = json.load(reader)["data"]
    def is_whitespace(c):
        if c == " " or c == "\t" or c == "\r" or c == "\n" or ord(c) == 0x202F:
            return True
        return False
    for entry in sfu_data:
        for paragraph in entry["paragraphs"]:
            paragraph_text = paragraph["context"]
            sfu_tokens = []
            prev_is_whitespace = True
            for c in paragraph_text:
                if is_whitespace(c):
                    prev_is_whitespace = True
                else:
                    if prev_is_whitespace:
                        sfu_tokens.append(c)
                    else:
                        sfu_tokens[-1] += c
                    prev_is_whitespace = False
            for qa in paragraph["qas"]:
                sfu_question_text = qa["question"]
                sfu_answer_text = qa["answers"][0]["text"]
                break

        break
    
sfu_context_text = ' '.join(sfu_tokens)

print("First question-answer pair: \n")
print("Question: " + str(sfu_question_text) + "\n")
print("Context: " + str(sfu_context_text) + "\n")
print("Answers: " + str(sfu_answer_text))


First question-answer pair: 

Question: How much research income did Simon Fraser University receive in 2017?

Context: In 2017, Simon Fraser University received a sponsored research income (external sources of research funds) of C$138.964 million, the 17th highest in Canada. In the same year, the university's faculty averaged a sponsored research income of $156,300, while graduates averaged $30,900.[37]

Answers: C$138.964 million


 ## 3. Approach



After research, we have found there are three popular models, BERT, ALBERT and DistilBERT, which are publicly used in QA system. From the understanding of their research papers [Reference d,e,f], we have summarized the advantages of each model 
BERT
ALBERT
DistilBERT

### 3.1 Baseline Model
First, we would like to learn and get the result from the above models with SQuAD, as we have not used the models before the project. Therefore, we have found a Github implementation on SQuAD dataset for transformer models [Reference a] as our baseline model, so we can start to improve and understand the models.

After learning the baseline for SQuAD, we are also interested to use the fine tune model them on different dataset, Google natural question (NQ) dataset [Reference c]. By using natural question, we only consider finding short answer from the context, and we have set the long answer as the context, which allows us to create fill the input lower than the maximum sequeuce limit 512 tokens. Therefore, we have create function to filter any question pair which does not have a short answer in Google NQ, and create the function (read_nq_examples in section 4) to read the NQ data instead of SQuAD data.


And we also test with our own created SFU dataset which includes 100 questions related to SFU, and the Wikipedia answer. The SFU dataset would not be part of the train data, but only for getting prediction using the fine tuned model. 

All the dataset structure would indicated under Data Section.

### 3.2 Improved Model
Secondly, we also want to improve the prediction accuracy with using data augmentation in pre-processing and ensemble method in post-processing.

#### 3.2.1 Pre-Processing
For data augmentation, we think that if we use the ground true answer inside dataset which may not be enough for the model, if the prediction produced the synonyms words of the ground true answer. Therefore, we desire to replace the ground true answer with synonyms words. We hope that it could improve our accuracy comparing with the baseline result.

#### 3.2.2 Post-Processing

We also applied several post-processing algorithms, specifically different types of ensemble methods, to combine the prediction outputs from chosen models together. To do so, we first analysis the individual model performance on SQuAD dataset, and choose the mean F1 score as the measurement for how good a model is. We list the measurement below.

{

    "bert":(88.26+88.26+88.08)/3,
    "albert":(88.85+89.38+89.40)/3,
    "distilbert":(85.03+85.50+85.21)/3,
    "albertAug":89.46,
    "albertGoogle":89.46

}

1. Stochastic: ensemble_stochastic.py takes the .json files containing the best predictions from selected models, and ensemble them using a stochastic approach. It works as follows: For the same question, if there is a disagreement between the predictions, the model with a better mean F1 score prevails at probability T, where T is a threshold chosen between 0.5 to 1 (we did experiments with 0.85, 0.9, 0.95, 0.98). The motivation it to squeeze out the performance if a model with low overall performance does outperform its counterparts. We are inspired by ideas from this paper(https://web.stanford.edu/class/cs224n/reports/default/15792214.pdf)


2. Majority vote: ensemble_majority_vote.py takes the .json files containing nbest (we choose n = 20) predictions from selected models, and ensemble them using a voting mechanism. For each question, it counts how many times one specific answer occurs in the nbest predictions from all selected models. For each unique answer, every time it occurs in a model’s nbest predictions, it receive vote V, where we choose V as the mean F1 score of that model. It sums up all the votes each answer receive, and choose the one with maximum votes as the best prediction.


3. Weighted probability sum: ensemble_probability_sum.py takes the .json files containing nbest (we choose n = 20) predictions from selected models, and ensemble them according the weighted sum of probability. For each unique answer to a question, every time it occurs in a model’s nbest predictions, it receive score P * V, where we choose P as the probability associated with that prediction, and V as the mean F1 score of that model. It sums up all the scores each answer receive, and choose the one with maximum score as the best prediction. We are inspired by ideas from this paper (https://web.stanford.edu/class/cs224n/reports/default/15845024.pdf)




 ### 4. Code

Code from open-source project:

 1. We used pretrained BERT, ALBERT, and DistilBERT models from  https://huggingface.co/transformers/pretrained_models.html as well as the fine-tuning script they provide. 
 
 
 2. We also use evaluate-v1.1.py from offical SQuAD https://rajpurkar.github.io/SQuAD-explorer/ to compare prdictions aganist ground truth.

The following code are created by us:

#### 4.1 Pre-Processing Code

##### 4.1.1 This code will shows how we load Google NQ dataset to the baseline model

```python
print("This may take some time to read all the samples")
# modified read_squad_examples to read Google NQ Dataset     
def read_nq_examples(input_file, is_training):
    """Read a Google NQ json file into a list of SquadExample."""
    def is_whitespace(c):
        if c == " " or c == "\t" or c == "\r" or c == "\n" or ord(c) == 0x202F:
            return True
        return False
    examples = []
    is_impossible = False
    use_cached = False
    if(use_cached): 
        examples = joblib.load('cached_example.sav')
    else: 
        with open(input_file, "r", encoding='utf-8') as reader:
            for line in reader:
                input_data = json.loads(line)
                paragraph_text = input_data["document_text"]
                doc_tokens = []
                char_to_word_offset = []
                prev_is_whitespace = True
                for c in paragraph_text:
                    if is_whitespace(c):
                        prev_is_whitespace = True
                    else:
                        if prev_is_whitespace:
                            doc_tokens.append(c)
                        else:
                            doc_tokens[-1] += c
                        prev_is_whitespace = False
                    char_to_word_offset.append(len(doc_tokens) - 1)
                start_position = None
                end_position = None
                orig_answer_text = None
                qas_id = input_data["example_id"]
                question_text = input_data["question_text"]
                if is_training:
                    long_answer = input_data["annotations"][0]["long_answer"]
                    short_answers = input_data["annotations"][0]["short_answers"]
                    if (len(short_answers) < 1 and not is_impossible):
                        raise ValueError(
                                    "For training, each question should have exactly 1 answer.")
                    short_answer = short_answers[0]
                    context_text = doc_tokens[long_answer["start_token"]:long_answer["end_token"]]

                    start_position = short_answer["start_token"] - long_answer["start_token"]
                    end_position = short_answer["end_token"] - long_answer["start_token"]

                    non_html_tokens = []
                    html_index = []
                    count = 0
                    for token in context_text:
                        html_index.append(count)
                        if (re.match(r'<[^>]+>', token)):
                            count = count + 1
                            continue
                        else:
                            non_html_tokens.append(token)
                    
                    no_html_paragraph_text= ' '.join(non_html_tokens)
                    answer = short_answers[0]
                    orig_answer_text = " ".join(doc_tokens[answer["start_token"]:answer["end_token"]])

                    start_position = start_position - html_index[start_position]
                    end_position = end_position -  html_index[end_position]

                    new_answer = " ".join(non_html_tokens[start_position:end_position])
                    # we would use the same format as SquadExample here
                    example = SquadExample(
                            qas_id=qas_id,
                            question_text=question_text,
                            doc_tokens=non_html_tokens,
                            orig_answer_text=new_answer,
                            start_position=start_position,
                            end_position=end_position,
                            is_impossible=is_impossible)
                    examples.append(example)

        logger.info("Number of examples %s", len(examples))
        joblib.dump((examples), 'cached_example.sav')  
    return examples
read_nq_examples('nq-short-train.json',is_training=True)
```

##### 4.1.2 This code will shows how we implements the data augmentation with synonyms words using Wordnet on SQuAD v1.1 dataset

In [57]:
import nltk
from nltk.corpus import wordnet 
nltk.download('wordnet')

def replace_synonyms_words(word):
    # Then, we're going to use the term "program" to find synsets like so: 
    synonyms_word = wordnet.synsets(word) 
    if(len(synonyms_word) < 2):
        return word
    else:
        new_word = synonyms_word[1].lemmas()[0].name()
        return new_word

def read_squad_examples(input_file, is_training, version_2_with_negative):
    """Read a SQuAD json file into a list of SquadExample."""
    count = 0
    with open(input_file, "r", encoding='utf-8') as reader:
        input_data = json.load(reader)["data"]

    def is_whitespace(c):
        if c == " " or c == "\t" or c == "\r" or c == "\n" or ord(c) == 0x202F:
            return True
        return False

    examples = []
    for entry in input_data:
        for paragraph in entry["paragraphs"]:
            paragraph_text = paragraph["context"]
            doc_tokens = []
            char_to_word_offset = []
            prev_is_whitespace = True
            for c in paragraph_text:
                if is_whitespace(c):
                    prev_is_whitespace = True
                else:
                    if prev_is_whitespace:
                        doc_tokens.append(c)
                    else:
                        doc_tokens[-1] += c
                    prev_is_whitespace = False
                char_to_word_offset.append(len(doc_tokens) - 1)
        
            for qa in paragraph["qas"]:
                qas_id = qa["id"]
                question_text = qa["question"]
                start_position = None
                end_position = None
                orig_answer_text = None
                is_impossible = False
                if is_training:
                    if version_2_with_negative:
                        is_impossible = qa["is_impossible"]
                    if (len(qa["answers"]) != 1) and (not is_impossible):
                        raise ValueError(
                            "For training, each question should have exactly 1 answer.")
                    if not is_impossible:
                        answer = qa["answers"][0]
                        orig_answer_text = answer["text"]
                        answer_offset = answer["answer_start"]
                        answer_length = len(orig_answer_text)
                        start_position = char_to_word_offset[answer_offset]
                        end_position = char_to_word_offset[answer_offset + answer_length - 1]

                        actual_text = " ".join(doc_tokens[start_position:(end_position + 1)])
                        cleaned_answer_text = " ".join(
                            whitespace_tokenize(orig_answer_text))
                        if actual_text.find(cleaned_answer_text) == -1:
                            logger.warning("Could not find answer: '%s' vs. '%s'",
                                           actual_text, cleaned_answer_text)
                            continue
                        if count < len(input_data) * 0.7:
                            text = doc_tokens[start_position:(end_position + 1)]
                            text[0] = replace_synonyms_words(text[0])
                            orig_answer_text = " ".join(text)
                        example = SquadExample(
                            qas_id=qas_id,
                            question_text=question_text,
                            doc_tokens=doc_tokens,
                            orig_answer_text=orig_answer_text,
                            start_position=start_position,
                            end_position=end_position,
                            is_impossible=is_impossible)
                        examples.append(example)
                    else:
                        start_position = -1
                        end_position = -1
                        orig_answer_text = ""

                example = SquadExample(
                    qas_id=qas_id,
                    question_text=question_text,
                    doc_tokens=doc_tokens,
                    orig_answer_text=orig_answer_text,
                    start_position=start_position,
                    end_position=end_position,
                    is_impossible=is_impossible)
                examples.append(example)
                

                
    return examples
read_squad_examples('train-v1.1.json', True, False)

[nltk_data] Downloading package wordnet to /home/cyrus-
[nltk_data]     gpu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


[qas_id: 5733be284776f41900661182, question_text: To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?, doc_tokens: [Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.], start_position: 90, end_position: 92,
 qas_id: 5733be284776f41900661182, question_text: To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?, doc_t

#### 4.2 Post-Processing Code

We wrote three ensemble algorithms as described in section 3.2.2 by ourself. We list them here only for reference. Please copy and paste them into python file or download our source.zip to run them locally.

##### 4.2.1 ensemble_stochastic.py

To run
1. download and install pyspark (pip install pyspark)
2. ```spark-submit ensemble_stochastic.py [directory that only contains *_best_prodiction.json]```

```python
from pyspark import SparkConf, SparkContext
import sys, json, glob, random, os
assert sys.version_info >= (3, 5) # make sure we have Python 3.5+

"{ model_name : avg F1 score on SQUAD}"
model_performance = {
    "bert":(88.26+88.26+88.08)/3,
    "albert":(88.85+89.38+89.40)/3,
    "distilbert":(85.03+85.50+85.21)/3,
    "albertAug":89.46,
    "albertGoogle":89.46
    }

threshold = 0.85

def chose_best_answer(x, y):
    x_model = x[0]
    y_model = y[0]

    # 80% chance to chose answer predicted from high performance model
    if (model_performance[x_model] > model_performance[y_model]) and random.random() < threshold:
        return x
    else:
        return y

def main(inputs, output):
    predictions_list = []
    os.chdir(inputs)
    for filename in glob.glob('*.json'):
        "assume predictions file format as bert_19000.json"
        mode_name = filename.split("_")[0]
        with open(filename, 'r') as f:
            pred = json.load(f)
            for key in pred:
                predictions_list.append((key,(mode_name,pred[key])))

    preds_rdd = sc.parallelize(predictions_list)
    best_ans_rdd = preds_rdd.reduceByKey(chose_best_answer)
    # best_ans = best_ans_rdd.sortBy(lambda x: int(x[0].split("\"")[0])).collect()
    best_ans = best_ans_rdd.sortByKey().collect()
    out_dict = {}
    for row in best_ans:
        out_dict[row[0]] = row[1][1]

    with open(output, 'w') as json_file:
        json.dump(out_dict, json_file)

if __name__ == '__main__':
    conf = SparkConf().setAppName('ensemble predictions')
    sc = SparkContext(conf=conf)
    sc.setLogLevel('WARN')
    assert sc.version >= '2.4'  # make sure we have Spark 2.4+
    inputs = sys.argv[1]
    output = "ensemble_stochastic_"+ str(threshold)+".json"
    main(inputs,output)
```

##### 4.2.2 ensemble_majority_vote.py

To run:

```python3 ensemble_majority_vote.py [directory that only contains *_nbest_prodiction.json]```

```python
import sys, json, glob, random, os
assert sys.version_info >= (3, 5) # make sure we have Python 3.5+

"{ model_name : avg F1 score on SQUAD}"
model_performance = {
    "bert":(88.26+88.26+88.08)/3,
    "albert":(88.85+89.38+89.40)/3,
    "distilbert":(85.03+85.50+85.21)/3,
    "albertAug":89.46,
    "albertGoogle":89.46
    }

def main(inputs, output): 
    all_nbest_preds = {}
    best_preds = {}

    # 1. load all nbest prediction file from every models
    os.chdir(inputs)
    for filename in glob.glob('*.json'):
        "assume predictions file format as bert_19000.json"
        mode_name = filename.split("_")[0]
        with open(filename, 'r') as f:
            nbest_pred = json.load(f)
            for question_id in nbest_pred:
                for ans in nbest_pred[question_id]:
                    ans["model"] = mode_name

                if question_id in all_nbest_preds:
                    # entend existing answer
                    all_nbest_preds[question_id].extend(nbest_pred[question_id])
                else:
                    all_nbest_preds[question_id] = nbest_pred[question_id]
    # 2. load original questions list

    # 3. iterate over all questions
    for qid in all_nbest_preds:
        ans_list = all_nbest_preds[qid]
        # count votes
        counts = {}
        for ans in ans_list:
            counts[ans["text"]] = counts.get(ans["text"], 0) + model_performance[ans["model"]]

        # select most voted answer
        best_ans = max(counts.items(), key=lambda x: x[1])[0]
        best_preds[qid] = best_ans

    # output best answer
    with open(output, 'w') as json_file:
        json.dump(best_preds, json_file)

if __name__ == '__main__':
    inputs = sys.argv[1]
    output = "ensemble_majority_vote.json"
    main(inputs,output)
```

##### 4.2.1 ensemble_probability_sum.py

To run:

```python3 ensemble_probability_sum.py [directory that only contains *_nbest_prodiction.json]```

```python
import sys, json, glob, random, os
assert sys.version_info >= (3, 5) # make sure we have Python 3.5+

"{ model_name : avg F1 score on SQUAD}"
model_performance = {
    "bert":(88.26+88.26+88.08)/3,
    "albert":(88.85+89.38+89.40)/3,
    "distilbert":(85.03+85.50+85.21)/3,
    "albertAug":89.46,
    "albertGoogle":89.46
    }

def main(inputs, output): 
    all_nbest_preds = {}
    best_preds = {}

    # 1. load all nbest prediction file from every models
    os.chdir(inputs)
    for filename in glob.glob('*.json'):
        "assume predictions file format as bert_19000.json"
        mode_name = filename.split("_")[0]
        with open(filename, 'r') as f:
            nbest_pred = json.load(f)
            for question_id in nbest_pred:
                for ans in nbest_pred[question_id]:
                    ans["model"] = mode_name

                if question_id in all_nbest_preds:
                    # entend existing answer
                    all_nbest_preds[question_id].extend(nbest_pred[question_id])
                else:
                    all_nbest_preds[question_id] = nbest_pred[question_id]
    # 2. load original questions list

    # 3. iterate over all questions
    for qid in all_nbest_preds:
        ans_list = all_nbest_preds[qid]
        # count votes
        counts = {}
        for ans in ans_list:
            counts[ans["text"]] = counts.get(ans["text"], 0) + ans["probability"] * model_performance[ans["model"]]

        # select most voted answer
        best_ans = max(counts.items(), key=lambda x: x[1])[0]
        best_preds[qid] = best_ans

    # output best answer
    with open(output, 'w') as json_file:
        json.dump(best_preds, json_file)

if __name__ == '__main__':
    inputs = sys.argv[1]
    output = "ensemble_probability_sum.json"
    main(inputs,output)
```

 ### 5. Experimental Setup

#### 5.1 Evaluation Metric

In this project, we adopt the same evaluation metrics for SQuAD 1.1, a popular question answering dataset. The metrics include:

1. Exact match. This metric measures the percentage of predictions that match any one of the ground truth answers exactly.


2. F1 score. This metric measures the average overlap between the prediction and ground truth answer. We treat the prediction and ground truth as bags of tokens, and compute their F1. We take the maximum F1 over all of the ground truth answers for a given question, and then average over all of the questions.

#### 5.2 Run Prediction

We use predict_albert.py, predict_albert.py, predict_albert.py to run predictions on test data. Due to the size of the model weights, predction for test data is not run here. You can download our trained mdoel weights from this link: 

https://drive.google.com/open?id=1UQ22LQK8mt9LqqHsFkmUN6NI2yIGaDnP

To do prediction, run command like

```python3 predict_albert.py [directory of your model weights] [directory of your test data]```

#### 5.3 Evaluation Process

After prediction, we get the prediction results, i.e, the answers predicted by different models. Then we use evaluate-v1.1.py to show F1 and EM score with respect to the reference. Here is an evaluation process where we get the highest F1 and EM score for the Ens10 model:

First download the prediction results：

In [64]:

!wget --no-check-certificate -O ens10.json "https://onedrive.live.com/download?cid=1ED3E57B6F717CEC&resid=1ED3E57B6F717CEC%21123638&authkey=ALvK9fnAZtobjO4"

--2019-12-08 23:40:10--  https://onedrive.live.com/download?cid=1ED3E57B6F717CEC&resid=1ED3E57B6F717CEC%21123638&authkey=ALvK9fnAZtobjO4
Resolving onedrive.live.com (onedrive.live.com)... 13.107.42.13
Connecting to onedrive.live.com (onedrive.live.com)|13.107.42.13|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://qg99ng.by.files.1drv.com/y4m5Fndl2Ym6GYywdVM7CZYclD2ss2BU-_O7hI09QAlMpHniDvXHQP5iXz_ru0ouZHl6mIPRxL2sW3ft9liKZ4dDDaud7FD-fMqeBdi7nd5MXASp2uA5mZdik5kP3DdEVBmPErDNwc1EH2zcussJtTKi9sdIRVRlab9ppwEai_RxRUPdyeQiPctuWho7XePb0bS9EWz3Kus5i6yn9aZYQNOdQ/ens10.json?download&psid=1 [following]
--2019-12-08 23:40:11--  https://qg99ng.by.files.1drv.com/y4m5Fndl2Ym6GYywdVM7CZYclD2ss2BU-_O7hI09QAlMpHniDvXHQP5iXz_ru0ouZHl6mIPRxL2sW3ft9liKZ4dDDaud7FD-fMqeBdi7nd5MXASp2uA5mZdik5kP3DdEVBmPErDNwc1EH2zcussJtTKi9sdIRVRlab9ppwEai_RxRUPdyeQiPctuWho7XePb0bS9EWz3Kus5i6yn9aZYQNOdQ/ens10.json?download&psid=1
Resolving qg99ng.by.files.1drv.com (qg99ng.by.files.1drv.com).

Then run evaluate-v1.1.py to get the F1 and EM score:

In [1]:
!python3 ../output/evaluate-v1.1.py ../output/reference/sfu.json ../output/predictions/sfu/ens/ens10.json

{"exact_match": 89.0, "f1": 95.70760851349085}


Next section will show our comparison between different models:

1. We first compared the performance of individual models BERT, ALBERT and DistilBERT on SQuAD 1.1 and SFU dataset.

2. After getting some idea about how individual model perform, we try different ensemble methods to imporve the prediction by combining model with different characteristic.

The specific ensemble setting can be found in the result section.

 ### 6. Results

<font color="RED">You may need to read this results table in Python notebook, instead of gitlab. Gitlab will cause column misplaced</font>

| ID   | Model|Training Set|Learning Rate|Epochs| SQuAD F1 | SQuAD EM | SFU F1 | SFU EM | Pred. Time(s)/SFU Data |
|------|------|------|------|------|------|------|------|------|------|
| 1    | BERT |SQuAD| 3e-5 |  2   | 88.26| 80.73| 90.85| 84.00| -|
| 2    | BERT |SQuAD| 3e-5 |  3   | 88.26| 80.56| 91.67| 84.00| -|
| 3    | BERT |SQuAD| 3e-5 |  4   | 88.08| 79.72| 92.59| 87.00| 268.25|
| 4    |ALBERT|SQuAD| 3e-5 |  3   | 88.85| 80.85| 92.77| 84.00| -|
| 5    |ALBERT|SQuAD| 3e-5 |  4   | 89.38| 81.59| 93.47| 84.00| -|
| 6    |ALBERT|SQuAD| 3e-5 |  5   | 89.40| 81.61| 93.38| 83.00| 264.08 | 
| 7    |DistilBERT|SQuAD| 3e-5 |  2   | 85.03| 76.92| 91.49| 83.00| -|
| 8    |DistilBERT|SQuAD| 3e-5 |  3   | 85.50| 77.22| 90.12| 82.00| -|
| 9    |DistilBERT|SQuAD| 3e-5 |  4   | 85.21| 76.87| 89.26| 82.00| <font color=red>127.67<font>|
| 10    |ALBERT| SQuAD + data augmentation| 3e-5 |  4   |89.46  |81.84  |94.57 |85.00 | -|
| 11    |ALBERT| Google NQ | 3e-5 |  4   | 89.46 | 81.84 |94.57 |85.00 | -|
| Ens1 |<td colspan=3> 1 + 2 + 3 + 4 + 5 + 6 + 7 + 8 + 9 + 10 + 11 (Stochastic 0.85) | 89.38| 81.71| 94.57| 85.00| -|
| Ens2 |<td colspan=3> 1 + 2 + 3 + 4 + 5 + 6 + 7 + 8 + 9 + 10 + 11 (Stochastic 0.9) | 89.45| 81.77| 94.57| 85.00| -|
| Ens3 |<td colspan=3> 1 + 2 + 3 + 4 + 5 + 6 + 7 + 8 + 9 + 10 + 11 (Stochastic 0.95) | 89.42| 81.82| 94.57| 85.00| -|
| Ens4 |<td colspan=3> 1 + 2 + 3 + 4 + 5 + 6 + 7 + 8 + 9 + 10 + 11 (Stochastic 0.98) | 89.43| 81.80| 94.48| 85.00| -|
| Ens5 |<td colspan=3> 3 + 10 + 11 (Majority vote) |85.82 |74.39  |93.15 |84.00 | -|
| Ens6 |<td colspan=3> 5 + 6 + 10 + 11 (Majority vote) |89.43|81.57  |94.57  |85.00  | -|
| Ens7 |<td colspan=3> 1 + 2 + 3 + 4 + 5 (Majority vote) |85.89  |74.26  |87.79 | 76.00| -|
| Ens8 |<td colspan=3> 1 + 2 + 3 + 4 + 5 + 6 + 7 + 8 + 9 + 10 + 11 (Majority vote) |82.88  |68.81  |90.31 |81.00 | -|
| Ens9 |<td colspan=3> 5 + 6 + 10 + 11 (Weighted probability sum) | 89.85 | 82.51 |94.57 | 85.00| -|
| Ens10 |<td colspan=3> 1 + 2 + 3 + 4 + 5 + 6 + 7 + 8 + 9 + 10 + 11 (Weighted probability sum) |<font color=red>90.86</font>  |<font color=red>84.60<font>  |<font color=red>95.71 <font> |<font color=red>89.00 <font> | -|
| Ens11 |<td colspan=3> 1 + 2 + 3 + 4 + 5 + 6 + 7 + 8 + 9 + 10 (Weighted probability sum) |90.64  |84.29  |94.18 |86.00 | -|
| Ens12 |<td colspan=3> 1 + 2 + 3 + 4 + 5 + 6 + 7 + 8 + 9 (Weighted probability sum) |90.30  | 83.96 |94.21 |87.00 | -|
   

 ### 7. Analysis of the Results

#### 7.1 Overall Performance

We achieved improvement over our baseline, mainly because data augmentation and weighted probability sum ensemble works as we expected. 

From the result in section 6, we can see the following summary:

Based on the result from ID 1 – 9:

<ol> 1) DistilBERT has significantly reduced prediction time by 50%. However, it reduces the accuracy on F1 and EM.</ol>
<ol> 2) Albert has the better F1 and EM score by comparing BERT and DistilBERT</ol>

Based on the result from ID 10, 11:

<ol> 3) Increasing the training data size by either using a larger dataset or augumenting data can slightly improve the model performance</ol>
<ol> 4) Although trained on different dataset, 10 and 11 produce exactly the same prediction on both SQuADv1.1 and SFU test dataset.</ol>

Based on Ens1 – Ens12:

<ol> 5) Ens10, which used weighted probability sum from model 1 - 11, achieved the highest F1 and EM scores on both test set</ol>
<ol> 6) The predication time has increased significantly as we cascade the models for predictions </ol>

#### 7.2 Prediction Analysis

We look at different models' performance on the same question and context. Here is an example for one context and question:

Context:

In 2017, Simon Fraser University received a sponsored research income (external sources of research funds) of C\\$138.964 million, the 17th highest in Canada. In the same year, the university's faculty averaged a sponsored research income of \\$156,300, while graduates averaged $ 30,900.
    
Question:

How much research income did Simon Fraser University receive in 2017?
    
And the table below shows the different predictions given a model and different epoch number:

| ID   | Model|Epochs| Answer |
|------|------|------|------|
|  a  |BERT |2| C\\$138.964 million |
| b   |BERT |3| C\\$138.964 million |
|  c  | BERT|4|  C\\$138.964 million|
|   d | ALBERT|3| C\\$138.964 million, |
|  e  | ALBERT|4| C\\$138.964 million, |
|  f  |ALBERT |5|  C\\$138.964 million,|
|  g  |DistilBERT |2|  C\\$138.964 million|
|  h  |DistilBERT |3| sponsored research income (external sources of research funds) of C\\$138.964 million |
|  i  | DistilBERT|4|  sponsored research income (external sources of research funds) of C\\$138.964 million|

From the table, we observe three insights:
    
1. For BERT model, it precisely captures the number forms and the way of representating currency, which implies that this domain has already been well learn by BERT and that the training data including wikipedia contains similar information.

2. For ALBERT model, it captures number forms and the way of representating currency, but confuse with the punctuation. Though for the overall F1 and EM score, ALBERT outperforms BERT and DistilBERT, but in some cases, it has problems with respect to the ending index of the answer.

3. For DistilBERT model, at epoch 1, the model precisely captures the number forms and the way of representating currency. But at the epoch 2 and 3, the model gives a wrong answer with respect to the starting index. This suggests that DistilBERT may have overfitting issues when training epoch becomes large.

### 8. Conclusion
In conclusion, the project has meet our expectation, and answered three of our aimed research questions. The enesembles method allows us to increase the accuarcy of SQuAD by 1.46% on F1 and  2.99% on EM. Also, increase the accuary on SFU dataset by 2.33% on F1 and 6% on EM.

1. Neural QA model trained on generic dataset perform very well at answering SFU related questions.


2. Model performance increases consistently across all algorithms, because SFU Test Dataset mostly contains short answers focusing on time, names and locations


3. Our Ensemble models improve the overall performance for SFU Test Dataset, however, the prediction time increases significantly

### 9. Future Improvement


1. We also hope to try the models with SQuAD v2.0 and A Conversational Question Answering Challenge (CoQA) dataset. 


2. We also believe that linguistic knowledge method could also improve our accuaracy.


3. Other than Ensemble method, we also seek to find a better method which can improve accuaracy and computation time.


4. We want to find another improvement methond which can improve both accuaracy and prediction time

### 10. Reference

a. https://github.com/huggingface/transformers

b. https://ai.google.com/research/NaturalQuestions

c. https://rajpurkar.github.io/SQuAD-explorer/

d. Yang, Zhilin, et al. "DistilBERT, a distilled version of BERT: smaller, faster, cheaper and lighter." arXiv preprint arXiv:1910.01108 (2019).

e. Devlin, Jacob, et al. "Bert: Pre-training of deep bidirectional transformers for language understanding." arXiv preprint arXiv:1810.04805 (2018).

f. Lan, Zhenzhong, et al. "Albert: A lite bert for self-supervised learning of language representations." arXiv preprint arXiv:1909.11942 (2019). 